In [ ]:
# --------------------------------------------------
# If needed, install these once in your Synapse workspace:
%pip install azure-storage-blob==12.14.1 requests==2.28.2 yfinance==0.2.18
# --------------------------------------------------


In [8]:
import time
import re
import json
import logging
import requests
from datetime import datetime, timedelta
from requests.exceptions import HTTPError
from azure.storage.blob import BlobServiceClient
import urllib.parse
import urllib.request
from urllib.error import HTTPError

In [9]:
# ----------------------------
# 2) Map Ticker → Company Name (via yfinance)
# ----------------------------

ticker_to_name = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "NVDA": "NVIDIA Corporation",
    "GOOG": "Alphabet Inc.",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "Amazon.com, Inc.",
    "META": "Meta Platforms, Inc.",
    "BRK-B": "Berkshire Hathaway Inc. New",
    "AVGO": "Broadcom Inc.",
    "TSLA": "Tesla, Inc.",
    "WMT": "Walmart Inc.",
    "LLY": "Eli Lilly and Company",
    "JPM": "JP Morgan Chase & Co.",
    "V": "Visa Inc.",
    "UNH": "UnitedHealth Group Incorporated",
    "MA": "Mastercard Incorporated",
    "XOM": "Exxon Mobil Corporation",
    "COST": "Costco Wholesale Corporation",
    "NFLX": "Netflix, Inc.",
    "PG": "Procter & Gamble Company (The)",
    "ORCL": "Oracle Corporation",
    "JNJ": "Johnson & Johnson",
    "HD": "Home Depot, Inc. (The)",
    "ABBV": "AbbVie Inc.",
    "KO": "Coca-Cola Company (The)",
    "TMUS": "T-Mobile US, Inc.",
    "BAC": "Bank of America Corporation",
    "PM": "Philip Morris International Inc",
    "CRM": "Salesforce, Inc.",
    "CVX": "Chevron Corporation",
    "CSCO": "Cisco Systems, Inc.",
    "MCD": "McDonald's Corporation",
    "ABT": "Abbott Laboratories",
    "IBM": "International Business Machines",
    "PLTR": "Palantir Technologies Inc.",
    "LIN": "Linde plc",
    "WFC": "Wells Fargo & Company",
    "PEP": "Pepsico, Inc.",
    "MRK": "Merck & Company, Inc.",
    "GE": "GE Aerospace",
    "T": "AT&T Inc.",
    "VZ": "Verizon Communications Inc.",
    "ACN": "Accenture plc",
    "AXP": "American Express Company",
    "ISRG": "Intuitive Surgical, Inc.",
    "MS": "Morgan Stanley",
    "RTX": "RTX Corporation",
    "TMO": "Thermo Fisher Scientific Inc",
    "INTU": "Intuit Inc.",
    "NOW": "ServiceNow, Inc.",
    "PGR": "Progressive Corporation (The)",
    "BX": "Blackstone Inc.",
    "AMGN": "Amgen Inc.",
    "GS": "Goldman Sachs Group, Inc. (The)",
    "QCOM": "QUALCOMM Incorporated",
    "AMD": "Advanced Micro Devices, Inc.",
    "DIS": "Walt Disney Company (The)",
    "UBER": "Uber Technologies, Inc.",
    "ADBE": "Adobe Inc.",
    "BKNG": "Booking Holdings Inc. Common St",
    "TJX": "TJX Companies, Inc. (The)",
    "SPGI": "S&P Global Inc.",
    "CAT": "Caterpillar, Inc.",
    "SCHW": "Charles Schwab Corporation (The",
    "BSX": "Boston Scientific Corporation",
    "NEE": "NextEra Energy, Inc.",
    "DHR": "Danaher Corporation",
    "BLK": "BlackRock, Inc.",
    "TXN": "Texas Instruments Incorporated",
    "SYK": "Stryker Corporation",
    "UNP": "Union Pacific Corporation",
    "GILD": "Gilead Sciences, Inc.",
    "CMCSA": "Comcast Corporation",
    "HON": "Honeywell International Inc.",
    "VRTX": "Vertex Pharmaceuticals Incorpor",
    "DE": "Deere & Company",
    "LOW": "Lowe's Companies, Inc.",
    "PFE": "Pfizer, Inc.",
    "ADP": "Automatic Data Processing, Inc.",
    "BA": "Boeing Company (The)",
    "C": "Citigroup, Inc.",
    "FI": "Fiserv, Inc.",
    "AMAT": "Applied Materials, Inc.",
    "MMC": "Marsh & McLennan Companies, Inc",
    "CB": "Chubb Limited",
    "PANW": "Palo Alto Networks, Inc.",
    "LMT": "Lockheed Martin Corporation",
    "COP": "ConocoPhillips",
    "ETN": "Eaton Corporation, PLC",
    "MDT": "Medtronic plc.",
    "BMY": "Bristol-Myers Squibb Company",
    "AMT": "American Tower Corporation (REI",
    "SO": "Southern Company (The)",
    "ELV": "Elevance Health, Inc.",
    "SBUX": "Starbucks Corporation",
    "MO": "Altria Group, Inc.",
    "KKR": "KKR & Co. Inc.",
    "CME": "CME Group Inc.",
    "WELL": "Welltower Inc.",
    "CRWD": "CrowdStrike Holdings, Inc.",
    "DUK": "Duke Energy Corporation (Holdin",
    "WM": "Waste Management, Inc.",
    "ANET": "Arista Networks, Inc.",
    "ICE": "Intercontinental Exchange Inc.",
    "PLD": "Prologis, Inc.",
    "CI": "The Cigna Group",
    "KLAC": "KLA Corporation",
    "ADI": "Analog Devices, Inc.",
    "INTC": "Intel Corporation",
    "MDLZ": "Mondelez International, Inc.",
    "GEV": "GE Vernova Inc.",
    "AJG": "Arthur J. Gallagher & Co.",
    "CVS": "CVS Health Corporation",
    "LRCX": "Lam Research Corporation",
    "MCK": "McKesson Corporation",
    "SHW": "Sherwin-Williams Company (The)",
    "CTAS": "Cintas Corporation",
    "HCA": "HCA Healthcare, Inc.",
    "UPS": "United Parcel Service, Inc.",
    "AON": "Aon plc",
    "NKE": "Nike, Inc.",
    "ORLY": "O'Reilly Automotive, Inc.",
    "MU": "Micron Technology, Inc.",
    "APH": "Amphenol Corporation",
    "MCO": "Moody's Corporation",
    "TT": "Trane Technologies plc",
    "NOC": "Northrop Grumman Corporation",
    "CL": "Colgate-Palmolive Company",
    "RSG": "Republic Services, Inc.",
    "EQIX": "Equinix, Inc.",
    "DASH": "DoorDash, Inc.",
    "TDG": "Transdigm Group Incorporated",
    "GD": "General Dynamics Corporation",
    "FTNT": "Fortinet, Inc.",
    "MMM": "3M Company",
    "PH": "Parker-Hannifin Corporation",
    "APO": "Apollo Global Management, Inc.",
    "CDNS": "Cadence Design Systems, Inc.",
    "MSI": "Motorola Solutions, Inc.",
    "ABNB": "Airbnb, Inc.",
    "WMB": "Williams Companies, Inc. (The)",
    "ITW": "Illinois Tool Works Inc.",
    "ECL": "Ecolab Inc.",
    "ZTS": "Zoetis Inc.",
    "CMG": "Chipotle Mexican Grill, Inc.",
    "SNPS": "Synopsys, Inc.",
    "CEG": "Constellation Energy Corporatio",
    "REGN": "Regeneron Pharmaceuticals, Inc.",
    "COF": "Capital One Financial Corporati",
    "WDAY": "Workday, Inc.",
    "NEM": "Newmont Corporation",
    "PYPL": "PayPal Holdings, Inc.",
    "AZO": "AutoZone, Inc.",
    "MAR": "Marriott International",
    "PNC": "PNC Financial Services Group, I",
    "ROP": "Roper Technologies, Inc.",
    "APD": "Air Products and Chemicals, Inc",
    "KMI": "Kinder Morgan, Inc.",
    "USB": "U.S. Bancorp",
    "DELL": "Dell Technologies Inc.",
    "EOG": "EOG Resources, Inc.",
    "AFL": "AFLAC Incorporated",
    "BDX": "Becton, Dickinson and Company",
    "CPRT": "Copart, Inc.",
    "MNST": "Monster Beverage Corporation",
    "EMR": "Emerson Electric Company",
    "AEP": "American Electric Power Company",
    "TRV": "The Travelers Companies, Inc.",
    "SPG": "Simon Property Group, Inc.",
    "ADSK": "Autodesk, Inc.",
    "BK": "The Bank of New York Mellon Cor",
    "COR": "Cencora, Inc.",
    "PAYX": "Paychex, Inc.",
    "CSX": "CSX Corporation",
    "CARR": "Carrier Global Corporation",
    "OKE": "ONEOK, Inc.",
    "JCI": "Johnson Controls International",
    "ALL": "Allstate Corporation (The)",
    "RCL": "Royal Caribbean Cruises Ltd.",
    "HWM": "Howmet Aerospace Inc.",
    "PSA": "Public Storage",
    "FDX": "FedEx Corporation",
    "HLT": "Hilton Worldwide Holdings Inc.",
    "O": "Realty Income Corporation",
    "DLR": "Digital Realty Trust, Inc.",
    "NSC": "Norfolk Southern Corporation",
    "MET": "MetLife, Inc.",
    "FCX": "Freeport-McMoRan, Inc.",
    "KDP": "Keurig Dr Pepper Inc.",
    "CHTR": "Charter Communications, Inc.",
    "GWW": "W.W. Grainger, Inc.",
    "AIG": "American International Group, I",
    "EXC": "Exelon Corporation",
    "KMB": "Kimberly-Clark Corporation",
    "FICO": "Fair Isaac Corporation",
    "ROST": "Ross Stores, Inc.",
    "TFC": "Truist Financial Corporation",
    "PCAR": "PACCAR Inc.",
    "FAST": "Fastenal Company",
    "SLB": "Schlumberger N.V.",
    "D": "Dominion Energy, Inc.",
    "KR": "Kroger Company (The)",
    "AMP": "Ameriprise Financial, Inc.",
    "SRE": "DBA Sempra",
    "GM": "General Motors Company",
    "AXON": "Axon Enterprise, Inc.",
    "KVUE": "Kenvue Inc.",
    "CCI": "Crown Castle Inc.",
    "NXPI": "NXP Semiconductors N.V.",
    "TGT": "Target Corporation",
    "MSCI": "MSCI Inc.",
    "NDAQ": "Nasdaq, Inc.",
    "LHX": "L3Harris Technologies, Inc.",
    "PEG": "Public Service Enterprise Group",
    "VRSK": "Verisk Analytics, Inc.",
    "CTVA": "Corteva, Inc.",
    "EW": "Edwards Lifesciences Corporatio",
    "XEL": "Xcel Energy Inc.",
    "PWR": "Quanta Services, Inc.",
    "YUM": "Yum! Brands, Inc.",
    "ED": "Consolidated Edison, Inc.",
    "DFS": "Discover Financial Services",
    "PSX": "Phillips 66",
    "CMI": "Cummins Inc.",
    "HES": "Hess Corporation",
    "URI": "United Rentals, Inc.",
    "OTIS": "Otis Worldwide Corporation",
    "MPC": "Marathon Petroleum Corporation",
    "DHI": "D.R. Horton, Inc.",
    "TEL": "TE Connectivity plc",
    "F": "Ford Motor Company",
    "VST": "Vistra Corp.",
    "EA": "Electronic Arts Inc.",
    "FIS": "Fidelity National Information S",
    "TTWO": "Take-Two Interactive Software,",
    "FANG": "Diamondback Energy, Inc.",
    "PCG": "Pacific Gas & Electric Co.",
    "BKR": "Baker Hughes Company",
    "AME": "AMETEK, Inc.",
    "OXY": "Occidental Petroleum Corporatio",
    "GRMN": "Garmin Ltd.",
    "PRU": "Prudential Financial, Inc.",
    "TRGP": "Targa Resources, Inc.",
    "CBRE": "CBRE Group Inc",
    "GLW": "Corning Incorporated",
    "ETR": "Entergy Corporation",
    "KHC": "The Kraft Heinz Company",
    "SYY": "Sysco Corporation",
    "CTSH": "Cognizant Technology Solutions",
    "ACGL": "Arch Capital Group Ltd.",
    "VLO": "Valero Energy Corporation",
    "BRO": "Brown & Brown, Inc.",
    "WEC": "WEC Energy Group, Inc.",
    "HSY": "The Hershey Company",
    "HUM": "Humana Inc.",
    "STZ": "Constellation Brands, Inc.",
    "HIG": "The Hartford Insurance Group, I",
    "CSGP": "CoStar Group, Inc.",
    "VICI": "VICI Properties Inc.",
    "ODFL": "Old Dominion Freight Line, Inc.",
    "IDXX": "IDEXX Laboratories, Inc.",
    "WTW": "Willis Towers Watson Public Lim",
    "CAH": "Cardinal Health, Inc.",
    "GIS": "General Mills, Inc.",
    "VMC": "Vulcan Materials Company (Holdi",
    "RMD": "ResMed Inc.",
    "IT": "Gartner, Inc.",
    "LULU": "lululemon athletica inc.",
    "CNC": "Centene Corporation",
    "EBAY": "eBay Inc.",
    "MLM": "Martin Marietta Materials, Inc.",
    "EXR": "Extra Space Storage Inc",
    "A": "Agilent Technologies, Inc.",
    "EQT": "EQT Corporation",
    "LYV": "Live Nation Entertainment, Inc.",
    "WAB": "Westinghouse Air Brake Technolo",
    "IR": "Ingersoll Rand Inc.",
    "VTR": "Ventas, Inc.",
    "GEHC": "GE HealthCare Technologies Inc.",
    "AWK": "American Water Works Company, I",
    "TPL": "Texas Pacific Land Corporation",
    "AVB": "AvalonBay Communities, Inc.",
    "K": "Kellanova",
    "EFX": "Equifax, Inc.",
    "LEN": "Lennar Corporation",
    "TSCO": "Tractor Supply Company",
    "BR": "Broadridge Financial Solutions,",
    "RJF": "Raymond James Financial, Inc.",
    "DTE": "DTE Energy Company",
    "ROL": "Rollins, Inc.",
    "AEE": "Ameren Corporation",
    "XYL": "Xylem Inc.",
    "IQV": "IQVIA Holdings, Inc.",
    "ANSS": "ANSYS, Inc.",
    "DXCM": "DexCom, Inc.",
    "ROK": "Rockwell Automation, Inc.",
    "DAL": "Delta Air Lines, Inc.",
    "WRB": "W.R. Berkley Corporation",
    "PPL": "PPL Corporation",
    "EQR": "Equity Residential",
    "ALLY": "Ally Financial Inc.",
    "MANH": "Manhattan Associates, Inc.",
    "DOCS": "Doximity, Inc.",
    "CNM": "Core & Main, Inc.",
    "BRBR": "BellRing Brands, Inc.",
    "CELH": "Celsius Holdings, Inc.",
    "ATR": "AptarGroup, Inc.",
    "DTM": "DT Midstream, Inc.",
    "CACI": "CACI International, Inc.",
    "BWXT": "BWX Technologies, Inc.",
    "NBIX": "Neurocrine Biosciences, Inc.",
    "TOL": "Toll Brothers, Inc.",
    "MTZ": "MasTec, Inc.",
    "SEIC": "SEI Investments Company",
    "ORI": "Old Republic International Corp",
    "X": "United States Steel Corporation",
    "MEDP": "Medpace Holdings, Inc.",
    "SAIA": "Saia, Inc.",
    "JEF": "Jefferies Financial Group Inc.",
    "OGE": "OGE Energy Corp",
    "FYBR": "Frontier Communications Parent,",
    "PR": "Permian Resources Corporation",
    "FHN": "First Horizon Corporation",
    "CUBE": "CubeSmart",
    "CHE": "Chemed Corp",
    "SF": "Stifel Financial Corporation",
    "COHR": "Coherent Corp.",
    "AIT": "Applied Industrial Technologies",
    "ARMK": "Aramark",
    "SNX": "TD SYNNEX Corporation",
    "CRS": "Carpenter Technology Corporatio",
    "BLD": "TopBuild Corp.",
    "PRI": "Primerica, Inc.",
    "INGR": "Ingredion Incorporated",
    "OVV": "Ovintiv Inc. (DE)",
    "SSB": "SouthState Corporation",
    "G": "Genpact Limited",
    "CIEN": "Ciena Corporation",
    "MASI": "Masimo Corporation",
    "APPF": "AppFolio, Inc.",
    "ADC": "Agree Realty Corporation",
    "PLNT": "Planet Fitness, Inc.",
    "VNOM": "Viper Energy, Inc.",
    "NVT": "nVent Electric plc",
    "WMS": "Advanced Drainage Systems, Inc.",
    "MLI": "Mueller Industries, Inc.",
    "TTEK": "Tetra Tech, Inc.",
    "EGP": "EastGroup Properties, Inc.",
    "HRB": "H&R Block, Inc.",
    "CR": "Crane Company",
    "RRC": "Range Resources Corporation",
    "HLNE": "Hamilton Lane Incorporated",
    "NYT": "New York Times Company (The)",
    "AM": "Antero Midstream Corporation",
    "BERY": "Berry Global Group, Inc.",
    "FND": "Floor & Decor Holdings, Inc.",
    "DBX": "Dropbox, Inc.",
    "CBSH": "Commerce Bancshares, Inc.",
    "REXR": "Rexford Industrial Realty, Inc.",
    "BRX": "Brixmor Property Group Inc.",
    "LAD": "Lithia Motors, Inc.",
    "NNN": "NNN REIT, Inc.",
    "HALO": "Halozyme Therapeutics, Inc.",
    "DCI": "Donaldson Company, Inc.",
    "EXP": "Eagle Materials Inc",
    "GPK": "Graphic Packaging Holding Compa",
    "ENSG": "The Ensign Group, Inc.",
    "CHDN": "Churchill Downs, Incorporated",
    "WBS": "Webster Financial Corporation",
    "SKX": "Skechers U.S.A., Inc.",
    "AYI": "Acuity Inc.",
    "EXLS": "ExlService Holdings, Inc.",
    "MTSI": "MACOM Technology Solutions Hold",
    "WCC": "WESCO International, Inc.",
    "ROIV": "Roivant Sciences Ltd.",
    "GAP": "Gap, Inc. (The)",
    "HQY": "HealthEquity, Inc.",
    "RGEN": "Repligen Corporation",
    "RLI": "RLI Corp.",
    "DLB": "Dolby Laboratories",
    "PNFP": "Pinnacle Financial Partners, In",
    "EVR": "Evercore Inc.",
    "OLLI": "Ollie's Bargain Outlet Holdings",
    "MIDD": "The Middleby Corporation",
    "WAL": "Western Alliance Bancorporation",
    "CFR": "Cullen/Frost Bankers, Inc.",
    "NFG": "National Fuel Gas Company",
    "LNTH": "Lantheus Holdings, Inc.",
    "ESAB": "ESAB Corporation",
    "BIO": "Bio-Rad Laboratories, Inc.",
    "KD": "Kyndryl Holdings, Inc.",
    "VNO": "Vornado Realty Trust",
    "PSN": "Parsons Corporation",
    "UGI": "UGI Corporation",
    "CMA": "Comerica Incorporated",
    "FN": "Fabrinet",
    "AAON": "AAON, Inc.",
    "TTC": "Toro Company (The)",
    "FOUR": "Shift4 Payments, Inc.",
    "AXTA": "Axalta Coating Systems Ltd.",
    "WTFC": "Wintrust Financial Corporation",
    "CVLT": "Commvault Systems, Inc.",
    "ATI": "ATI Inc.",
    "POST": "Post Holdings, Inc.",
    "RRX": "Regal Rexnord Corporation",
    "LNW": "Light & Wonder, Inc.",
    "KNX": "Knight-Swift Transportation Hol",
    "KBR": "KBR, Inc.",
    "WH": "Wyndham Hotels & Resorts, Inc.",
    "WTS": "Watts Water Technologies, Inc.",
    "SATS": "EchoStar Corporation",
    "UFPI": "UFP Industries, Inc.",
    "WING": "Wingstop Inc.",
    "AN": "AutoNation, Inc.",
    "AA": "Alcoa Corporation",
    "UMBF": "UMB Financial Corporation",
    "IDA": "IDACORP, Inc.",
    "AGCO": "AGCO Corporation",
    "ALV": "Autoliv, Inc.",
    "STWD": "STARWOOD PROPERTY TRUST, INC. S",
    "FR": "First Industrial Realty Trust,",
    "ZION": "Zions Bancorporation N.A.",
    "AAL": "American Airlines Group, Inc.",
    "HIMS": "Hims & Hers Health, Inc.",
    "SSD": "Simpson Manufacturing Company,",
    "FAF": "First American Corporation (New",
    "PB": "Prosperity Bancshares, Inc.",
    "FBIN": "Fortune Brands Innovations, Inc",
    "JAZZ": "Jazz Pharmaceuticals plc",
    "BBWI": "Bath & Body Works, Inc.",
    "STAG": "Stag Industrial, Inc.",
    "ONB": "Old National Bancorp",
    "BRKR": "Bruker Corporation",
    "LPX": "Louisiana-Pacific Corporation",
    "LSCC": "Lattice Semiconductor Corporati",
    "NXT": "Nextracker Inc.",
    "FCN": "FTI Consulting, Inc.",
    "TREX": "Trex Company, Inc.",
    "CHH": "Choice Hotels International, In",
    "ONTO": "Onto Innovation Inc.",
    "MTG": "MGIC Investment Corporation",
    "R": "Ryder System, Inc.",
    "ESNT": "Essent Group Ltd.",
    "THG": "Hanover Insurance Group Inc",
    "VMI": "Valmont Industries, Inc.",
    "TMHC": "Taylor Morrison Home Corporatio",
    "GTLS": "Chart Industries, Inc.",
    "MSA": "MSA Safety Incorporated",
    "FLR": "Fluor Corporation",
    "SNV": "Synovus Financial Corp.",
    "HR": "Healthcare Realty Trust Incorpo",
    "SAIC": "Science Applications Internatio",
    "FLS": "Flowserve Corporation",
    "OSK": "Oshkosh Corporation (Holding Co",
    "VOYA": "Voya Financial, Inc.",
    "ALK": "Alaska Air Group, Inc.",
    "OLED": "Universal Display Corporation",
    "SLM": "SLM Corporation",
    "KEX": "Kirby Corporation",
    "FCFS": "FirstCash Holdings, Inc.",
    "NSA": "National Storage Affiliates Tru",
    "NEU": "NewMarket Corp",
    "SIGI": "Selective Insurance Group, Inc.",
    "BYD": "Boyd Gaming Corporation",
    "KNF": "Knife Riv Holding Co.",
    "MTN": "Vail Resorts, Inc.",
    "SRPT": "Sarepta Therapeutics, Inc.",
    "GATX": "GATX Corporation",
    "SLGN": "Silgan Holdings Inc.",
    "CHRD": "Chord Energy Corporation",
    "DINO": "HF Sinclair Corporation",
    "OPCH": "Option Care Health, Inc.",
    "ARW": "Arrow Electronics, Inc.",
    "SWX": "Southwest Gas Holdings, Inc.",
    "LANC": "Lancaster Colony Corporation",
    "HOMB": "Home BancShares, Inc.",
    "CROX": "Crocs, Inc.",
    "RMBS": "Rambus, Inc.",
    "CADE": "Cadence Bank",
    "LOPE": "Grand Canyon Education, Inc.",
    "LSTR": "Landstar System, Inc.",
    "WEX": "WEX Inc.",
    "JHG": "Janus Henderson Group plc",
    "MAT": "Mattel, Inc.",
    "MTDR": "Matador Resources Company",
    "TXNM": "TXNM Energy, Inc.",
    "NJR": "NewJersey Resources Corporation",
    "GNTX": "Gentex Corporation",
    "CYTK": "Cytokinetics, Incorporated",
    "POR": "Portland General Electric Co",
    "CMC": "Commercial Metals Company",
    "CHX": "ChampionX Corporation",
    "NXST": "Nexstar Media Group, Inc.",
    "CRUS": "Cirrus Logic, Inc.",
    "AMG": "Affiliated Managers Group, Inc.",
    "DAR": "Darling Ingredients Inc.",
    "UBSI": "United Bankshares, Inc.",
    "KRG": "Kite Realty Group Trust",
    "MKSI": "MKS Instruments, Inc.",
    "OGS": "ONE Gas, Inc."
}

In [11]:
# ----------------------------
# Configuration
# ----------------------------
CONN_STR = "DefaultEndpointsProtocol=https;AccountName=ba870container;AccountKey=key;EndpointSuffix=core.windows.net"
CONTAINER_NAME       = "bronze"

API_KEY              = "key"

SEARCH_URL         = "https://gnews.io/api/v4/search"

ARTICLES_PER_PAGE  = 50
REQUESTS_PER_SEC   = 6
DELAY              = 1.0 / REQUESTS_PER_SEC

MAX_REQUESTS       = 4000       # your budget
TOTAL_TICKERS      = len(ticker_to_name)
PAGES_PER_TICKER   = max(1, MAX_REQUESTS // TOTAL_TICKERS)

START_DATE         = "2021-01-01"
END_DATE           = datetime.utcnow().strftime("%Y-%m-%d")
FROM_ISO           = f"{START_DATE}T00:00:00Z"
TO_ISO             = f"{END_DATE}T23:59:59Z"

In [12]:
def throttle():
    time.sleep(DELAY)

def fetch_headlines(query, page=1):
    """Call /search with proper ISO dates and return articles list."""
    params = {
        "apikey": API_KEY,
        "q": query,
        "from": FROM_ISO,
        "to": TO_ISO,
        "lang": "en",
        "country": "us",
        "max": ARTICLES_PER_PAGE,
        "page": page
    }
    url = SEARCH_URL + "?" + urllib.parse.urlencode(params, safe='" ')
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as resp:
        data = json.loads(resp.read().decode("utf-8"))
    throttle()
    return data.get("articles", [])

def save_all_blob(records):
    """Overwrite single JSON blob in Bronze."""
    client = BlobServiceClient.from_connection_string(CONN_STR)
    blob = client.get_blob_client(container=CONTAINER_NAME, blob="all_tickers_headlines.json")
    blob.upload_blob(json.dumps(records), overwrite=True)

In [13]:
# --------------------------------------------------
# Ingestion Loop
# --------------------------------------------------
all_headlines = []
req_count = 0

for ticker, company in ticker_to_name.items():
    if req_count >= MAX_REQUESTS:
        break

    # query: ticker OR "company name"
    q = f'{ticker} OR "{company}"'

    for page in range(1, PAGES_PER_TICKER + 1):
        if req_count >= MAX_REQUESTS:
            break

        try:
            articles = fetch_headlines(q, page)
        except HTTPError as e:
            print(f"⚠️ {ticker} pg{page} failed: {e}")
            break

        if not articles:
            break

        # tag & collect
        for art in articles:
            all_headlines.append({
                "ticker":      ticker,
                "company":     company,
                "title":       art.get("title"),
                "description": art.get("description"),
                "url":         art.get("url"),
                "publishedAt": art.get("publishedAt")
            })

        req_count += 1

    print(f"{ticker}: pages fetched = {page if articles else page-1}")

print(f"Collected {len(all_headlines)} headlines with {req_count} requests")

# --------------------------------------------------
# Save to Bronze
# --------------------------------------------------
save_all_blob(all_headlines)
print("all_tickers_headlines.json updated in Bronze")

In [14]:
import json
import pandas as pd
from azure.storage.blob import BlobServiceClient

# Configuration —
CONN_STR = "DefaultEndpointsProtocol=https;AccountName=ba870container;AccountKey= key;EndpointSuffix=core.windows.net"
CONTAINER       = "bronze"
BLOB_NAME     = "all_tickers_headlines.json"

# 3) Download & parse
blob_svc      = BlobServiceClient.from_connection_string(CONN_STR)
blob_client   = blob_svc.get_blob_client(container=CONTAINER, blob=BLOB_NAME)
raw          = blob_client.download_blob().readall().decode("utf-8")
records      = json.loads(raw)



In [29]:
# 4) Quick inspection
print(f"Total records: {len(records)}")
print("Sample record:")
for k,v in records[34567].items():
    print(f"  {k}: {v}")

In [35]:
# 5) Convert to DataFrame
df = pd.DataFrame(records)
display(df.tail(100))

In [38]:
df         = pd.DataFrame(records)
counts     = df["ticker"].value_counts().reset_index()
counts.columns = ["ticker", "headline_count"]

# 5) Display the result
display(counts)

In [39]:
df         = pd.DataFrame(records)
ticker = "MSFT"
df_filter = df[df["ticker"] == ticker]
print(f"\nRecords for {ticker}: {len(df_filter)}")
display(df_filter.head())